In [1]:
import numpy as np
import matplotlib.pylab as plt

In [66]:
# espace d'etats
etats = np.array([1,2,3,4,5,6])

In [67]:
# la probabilite sous-jacente
pi = np.array([0.3, 0.2, 0.1, 0.1, 0.15, 0.15])

In [155]:
# la matrice orginale par rapport à laquelle pi est reversible
P = np.zeros((6,6))
P[0] = np.array([8,2,1,1,1.5,1.5])/15
P[1] = np.array([2,4,1,1,1,1])/10
P[2] = np.array([2,2,2,2,1,1])/10
P[3] = np.array([2,2,2,2,1,1])/10
P[4] = np.array([3,2,1,1,8,0])/15
P[5] = np.array([3,2,1,1,0,8])/15
print("P = ")
print(np.round(P,3))

P = 
[[0.533 0.133 0.067 0.067 0.1   0.1  ]
 [0.2   0.4   0.1   0.1   0.1   0.1  ]
 [0.2   0.2   0.2   0.2   0.1   0.1  ]
 [0.2   0.2   0.2   0.2   0.1   0.1  ]
 [0.2   0.133 0.067 0.067 0.533 0.   ]
 [0.2   0.133 0.067 0.067 0.    0.533]]


# Partie 1 : cyclique

In [69]:
# la fonction pour trouver la valeur Alpha dans lemme 3 donné un cycle
def alpha(pi,P,cycle):
    Pi = np.zeros((6,6))
    for i in range(6):
        Pi[i] = pi
        
    a = 1
    A = np.transpose(Pi) * P
    cycle = cycle + [cycle[0]]
    
    for i in range(len(cycle)-1):
        a = np.minimum(a, A[cycle[i],cycle[i+1]])
    return a

In [97]:
# la fonction pour trouver la matrice antisymetrique dans le lemme 3
def Q_antisym(pi,P,cycle):
    
    Q = np.zeros(P.shape)
    alph = alpha(pi,P,cycle)
    cycle = cycle + [cycle[0]]
    
    for i in range(len(cycle)-1):
        Q[cycle[i],cycle[i+1]] = alph/pi[cycle[i]]
        Q[cycle[i+1],cycle[i]] = - alph/pi[cycle[i+1]]
        
    return Q

In [98]:
# c1 et c2 sont disjoints
c1 = [0,1,5]
c2 = [2,3,4]
c3 = [0,1,2,3]
Q1 = Q_antisym(pi,P,c1)
Q2 = Q_antisym(pi,P,c2)
Q3 = Q_antisym(pi,P,c3)

In [99]:
P_new1 = np.abs(P + Q1)
P_new2 = np.abs(P_new1 + Q2)
P_new3 = np.abs(P + Q3)

In [100]:
print("P + Q_1 = ")
print(np.round(P_new1,3))

print("P + Q_1 + Q_2 = ")
print(np.round(P_new2,3))
np.dot(pi,P_new2) # pour verifier que P_new2 est une matrice de transition sous laquelle pi est invariante

print("P + Q_3 = ")
print(np.round(P_new3,3))

P + Q_1 = 
[[0.533 0.2   0.067 0.067 0.1   0.033]
 [0.1   0.4   0.1   0.1   0.1   0.2  ]
 [0.2   0.2   0.2   0.2   0.1   0.1  ]
 [0.2   0.2   0.2   0.2   0.1   0.1  ]
 [0.2   0.133 0.067 0.067 0.533 0.   ]
 [0.333 0.    0.067 0.067 0.    0.533]]
P + Q_1 + Q_2 = 
[[0.533 0.2   0.067 0.067 0.1   0.033]
 [0.1   0.4   0.1   0.1   0.1   0.2  ]
 [0.2   0.2   0.2   0.3   0.    0.1  ]
 [0.2   0.2   0.1   0.2   0.2   0.1  ]
 [0.2   0.133 0.133 0.    0.533 0.   ]
 [0.333 0.    0.067 0.067 0.    0.533]]
P + Q_3 = 
[[0.533 0.2   0.067 0.    0.1   0.1  ]
 [0.1   0.4   0.2   0.1   0.1   0.1  ]
 [0.2   0.    0.2   0.4   0.1   0.1  ]
 [0.4   0.2   0.    0.2   0.1   0.1  ]
 [0.2   0.133 0.067 0.067 0.533 0.   ]
 [0.2   0.133 0.067 0.067 0.    0.533]]


In [101]:
# etats = espace d'etats
# etat_n_0 = numero de l'etat 0
# N = la longeur de la chaine
def markov_chain (P, etats, etat_original, N):
    etats_index = range(len(etats))
    chain = [etat_original]
    
    for n in range(N):
        etat_avant = chain[-1]
        pr = P[etat_avant]
        etat_n = np.random.choice(etats_index, p = pr)
        chain.append(etat_n)
        
    return [etats[n] for n in chain]

In [106]:
# on prend m trajectoires independantes
m = 1000
# N est la longeur de chaque chaine
N = 1000

etat_original  = 2

In [107]:
# la chaine generee par P
Chains0 = [markov_chain(P, etats, etat_original, N) for i in range(m)]

# la chaine generee par P + Q_1
Chains1 = [markov_chain(P_new1, etats, etat_original, N) for i in range(m)]

# la chaine generee par P + Q_1 + Q_2
Chains2 = [markov_chain(P_new2, etats, etat_original, N) for i in range(m)]

# la chaine generee par P + Q_3
Chains3 = [markov_chain(P_new3, etats, etat_original, N) for i in range(m)]

# Partie 2 : acyclique

In [114]:
# la fonction pour trouver deux elements non nuls et return beta
def beta(pi,P):
    b = pi*np.diag(P)
    x = np.nonzero(b)[0]
    if len(x) >= 2:
        return int(x[0]),int(x[1]),np.minimum(b[x[0]],b[x[1]])
    else:
        return 0,0,0

In [116]:
# la fonction pour trouver la matrice dans Theoreme 3 donné un cycle
def P_antisym_beta(pi, P):
    
    while True:
        Q = np.zeros(P.shape)
        i, j, bet = beta(pi, P)
        if bet == 0:
            break
        Q[i,i] = - bet/pi[i]
        Q[j,j] = - bet/pi[j]
        Q[i,j] =  bet/pi[i]
        Q[j,i] =  bet/pi[j]
        P = P + Q

    return P

In [117]:
aP_new = P_antisym_beta(pi, P)
print("P_new_acyclic = ")
print(np.round(aP_new,3))

P_new_acyclic = 
[[0.    0.4   0.133 0.133 0.233 0.1  ]
 [0.6   0.    0.1   0.1   0.1   0.1  ]
 [0.4   0.2   0.    0.2   0.1   0.1  ]
 [0.4   0.2   0.2   0.    0.1   0.1  ]
 [0.467 0.133 0.067 0.067 0.    0.267]
 [0.2   0.133 0.067 0.067 0.267 0.267]]


In [119]:
Q3 = Q_antisym(pi, aP_new, c3)
aP_new1 = np.abs(aP_new + Q3)
np.round(aP_new1,3)

array([[0.   , 0.467, 0.133, 0.067, 0.233, 0.1  ],
       [0.5  , 0.   , 0.2  , 0.1  , 0.1  , 0.1  ],
       [0.4  , 0.   , 0.   , 0.4  , 0.1  , 0.1  ],
       [0.6  , 0.2  , 0.   , 0.   , 0.1  , 0.1  ],
       [0.467, 0.133, 0.067, 0.067, 0.   , 0.267],
       [0.2  , 0.133, 0.067, 0.067, 0.267, 0.267]])

In [125]:
# la chaine generee par P' = aP_new
Chains4 = [markov_chain(aP_new, etats, etat_original, N) for i in range(m)]

# la chaine generee par P' + Q_3
Chains5 = [markov_chain(aP_new1, etats, etat_original, N) for i in range(m)]

# Resultats

In [146]:
# la fonction f (on prend des polynomes ici)
def f(etats, func):
    if type(func) == int:
        return np.array(etats)** func
    elif func == "sin":
        return np.sin(np.array(etats))
    elif func == 'exp':
        return np.exp(np.array(etats))
    elif func == 'log':
        return np.log(np.array(etats))
    else:
        print("choose another function please")
        return 0

# experance de f sous pi
def pi_f(etats, pi, f, func = 1):
    return np.sum(f(etats,func) * pi)

In [147]:
def var_asym(etats, pi, chain, f, func = 1):
    n = len(chain)
    moyenne_est = np.mean(f(chain,func))
    moyenne_vraie = pi_f(etats, pi, f, func)
    return n*(moyenne_est - moyenne_vraie)**2

In [148]:
# les variances asymptotiques correspondant aux P, P+Q_1 et P+Q_1+Q_2 avec f(x) = func
def print_var(func):
    var0 = np.mean(np.array([var_asym(etats, pi, chain0 , f, func) for chain0 in Chains0]))
    var1 = np.mean(np.array([var_asym(etats, pi, chain1 , f, func) for chain1 in Chains1]))
    var2 = np.mean(np.array([var_asym(etats, pi, chain2 , f, func) for chain2 in Chains2]))
    var3 = np.mean(np.array([var_asym(etats, pi, chain3 , f, func) for chain3 in Chains3]))
    var4 = np.mean(np.array([var_asym(etats, pi, chain4 , f, func) for chain4 in Chains4]))
    var5 = np.mean(np.array([var_asym(etats, pi, chain5 , f, func) for chain5 in Chains5]))
    
#     print('f(x) = ' + func)
    print('var_asym_P = ' + str(var0))
    print('var_asym_P_new1 = ' + str(var1))
    print('var_asym_P_new2 = ' + str(var2))
    print('var_asym_P_new3 = ' + str(var3))
    print('var_asym_P_new4 = ' + str(var4))
    print('var_asym_P_new5 = ' + str(var5))
    
    return 0

In [152]:
print_var(1)

var_asym_P = 6.872519780219778
var_asym_P_new1 = 6.241800099900099
var_asym_P_new2 = 6.199616483516483
var_asym_P_new3 = 6.61073816183816
var_asym_P_new4 = 4.1135824175824185
var_asym_P_new5 = 3.7310383616383604


0

In [153]:
print_var(2)

var_asym_P = 339.7744510489511
var_asym_P_new1 = 301.48104545454544
var_asym_P_new2 = 298.07079370629367
var_asym_P_new3 = 324.51601748251744
var_asym_P_new4 = 219.8684435564436
var_asym_P_new5 = 199.62048551448555


0

In [149]:
print_var('exp')

var_asym_P = 47299.276019137236
var_asym_P_new1 = 41822.592412228885
var_asym_P_new2 = 40769.5157284459
var_asym_P_new3 = 45234.883299982146
var_asym_P_new4 = 29142.274051920194
var_asym_P_new5 = 26345.857700501034


0

In [150]:
print_var('log')

var_asym_P = 0.9192969858501442
var_asym_P_new1 = 0.8620574552721171
var_asym_P_new2 = 0.8547644098081351
var_asym_P_new3 = 0.8845399978026138
var_asym_P_new4 = 0.4588391177005689
var_asym_P_new5 = 0.4160522746613623


0